$~$

# IT Academy - Data Science Itinerary



$~$

## S9 T2: Regression Models
___


In [ ]:
#importing libraries


$~$
___
####  Exercise 1

$~$

Create at least three different regression models to try to predict DelayedFlights.csv's flight delay (ArrDelay) as best you can. 

$~$
___

$~$
___
####  Exercise 2

$~$

Compare them based on MSE and R2.

$~$
___

$~$
___
####  Exercise 3

$~$

Train them using the different parameters they support.

$~$
___

$~$
___
####  Exercise 4

$~$

Compare your performance using the train / test approach or using all data (internal validation)

$~$
___

$~$
___
####  Exercise 4

$~$

Compare your performance using the train / test approach or using all data (internal validation)

$~$
___

$~$
___
####  Exercise 5

$~$

Perform some variable engineering process to improve prediction

$~$
___

$~$
___
####  Exercise 6

$~$

Do not use the DepDelay variable when making predictions


$~$
___

$~$
___
####  Conclusions

$~$




___
$~$

___
$~$

####  *References*:

+ []() 
+ []() 
+ []() 
$~$
___